# Read Future Climate Simulations

This script is used to read Raven model outputs after the model has been run with climate change projections. The script simply takes the output of the model and compares water balances (at the sub-watershed scale) for 2 user-defined time periods.

import packages

In [64]:
import pandas as pd
import numpy as np
import shapefile
from PIL import Image, ImageDraw

### Set model output directory, user settings

> Users must change the following input according to the model they wish to read.

In [65]:
ravenoutputdir = '../model_future_climate_CSIRO-Mk3-6-0_RCP85/output/' # directory of Raven output
hrufp = '../model_baseline/Raven2025-hruid.bil' # HRU reference map, produced using update_landuse.ipynb
swsfp = '../GIS/CH_Subs_Raven2025.shp' # Raven sub-basin polygons
mdlprfx = 'Raven2025' # model short-name prefix

## Import Sub-Basins

- Collect raster cells contained within each sub-basin

In [66]:
def polygonToCellIDs(pts, nrows=719, ncols=726, xul=1323330, yul=11906070, cs=60):
    img = Image.new('L', (ncols, nrows), 0)
    offset = [xul, yul-nrows*cs]
    pixelpoly = [((x-offset[0])/cs, nrows-(y-offset[1])/cs) for x,y in pts]
    ImageDraw.Draw(img).polygon(pixelpoly, outline=1, fill=1)
    mask = np.array(img)
    return np.arange(nrows*ncols)[mask.reshape(nrows*ncols)>0]    

def loadSWS(fp):
    sf = shapefile.Reader(fp)
    geom = sf.shapes()
    attr = sf.records()
    xr = dict()
    for i in range(len(geom)):
        sid = int(attr[i].SubId)
        xr[sid] = polygonToCellIDs(geom[i].points)
    return xr

In [67]:
xsb = loadSWS(swsfp) # sub-basin to cell cross reference

## Import HRU ID raster

In [68]:
# grid definition (keep consistent with input raster definition)
nrows = 719
ncols = 726
xul = 1323330
yul = 11906070
cs = 60. # grid cell width

# Reads integer rasters
def readIntRaster(fp):
    aa = np.fromfile(fp,np.int32)
    if len(aa) == nrows*ncols:
        x = dict(zip(np.arange(nrows*ncols),aa))
    elif len(aa) == nrows*ncols/2:
        aa = np.fromfile(fp,np.int16)
        x = dict(zip(np.arange(nrows*ncols),aa))
    else:
        print(" ** Warning, unknown raster size {}",fp)
        return None
    return x

In [69]:
xhru = readIntRaster(hrufp) # hruid to MODFLOW cell cross-reference

## Import Model Outputs

- Precipitation
- Total actual evapotranspiration (AET)
- Runoff
- Groundwater recharge
- Baseflow

### Model output file read function

In [ ]:
def readRaven(fp, isaccumulated, dtCurrentBegin='1991-10-01', dtCurrentEnd='2021-09-30', dtFutureBegin='2041-10-01', dtFutureEnd='2071-09-30'):
    # load data
    df = pd.read_csv(fp, skiprows=1, parse_dates=['month'], date_format="%Y-%m")
    df = df.iloc[:, :-1] # drop last column (..a Raven quirk)
    df = df.drop(columns='time')

    # change accumulation to discrete monthly values
    if isaccumulated:
        df2 = df.diff()
        df2 = df2.fillna(df)
        df2['month'] = df['month']
        df = df2    
    
    dfcurrent = df[(df.month>=dtCurrentBegin) & (df.month<=dtCurrentEnd)]
    dffuture =  df[(df.month>=dtFutureBegin) & (df.month<=dtFutureEnd)]

    def reformat(df1):
        # rename columns
        cols = [w.replace('mean.','').replace('cumulsum.','') for w in list(df.columns)]
        cols[1] = '0'
        df1.columns = cols
        df1.index=df1['month']
        df1 = df1.drop(columns='month')

        df1 = df1.mean(axis=0)*12
        return {int(k)+1:v for k,v in df1.items()}
    
    return reformat(dfcurrent), reformat(dffuture)

In [71]:
dprecip, dprecipFuture = readRaven(ravenoutputdir+mdlprfx+'_PRECIP_Monthly_CumulSum_ByHRU.csv',False)                             # precipitation
daet, daetFuture = readRaven(ravenoutputdir+mdlprfx+'_AET_Monthly_CumulSum_ByHRU.csv',False)                                      # evapotranspiration
dro, droFuture = readRaven(ravenoutputdir+mdlprfx+'_RUNOFF_Monthly_CumulSum_ByHRU.csv',False)                                     # runoff
drecharge, drechargeFuture = readRaven(ravenoutputdir+mdlprfx+'_BETWEEN_SOIL[1]_AND_SOIL[2]_Monthly_Average_ByHRU.csv',True)      # recharge
dbasflw, dbasflwFuture = readRaven(ravenoutputdir+mdlprfx+'_BETWEEN_SOIL[2]_AND_SURFACE_WATER_Monthly_Average_ByHRU.csv',True)    # baseflow

### CH sub-watershed summaries

Long-term averages, exported to csv file, by sub-watershed ID in mm/yr

In [72]:
def gatherHRUtoSWS(seriesname,dpr,dae,droff,dbf,dg):
    pre = dict()
    aet = dict()
    roff = dict()
    rch = dict()
    for c,h in xhru.items():
        if h==-9999: continue
        pre[c]=dpr[h]
        aet[c]=dae[h]
        roff[c]=droff[h]-dbf[h]
        rch[c]=dg[h]

    sp, sa, sr, sg = dict(), dict(), dict(), dict(),
    for sid, cids in xsb.items():
        sp[sid] = 0.
        sa[sid] = 0.
        sr[sid] = 0.
        sg[sid] = 0.
        for c in cids:
            if not c in pre: continue
            sp[sid] += pre[c]
            sa[sid] += aet[c]
            sr[sid] += roff[c]
            sg[sid] += rch[c]
        sp[sid] /= len(cids)
        sa[sid] /= len(cids)
        sr[sid] /= len(cids)
        sg[sid] /= len(cids)        

    df = pd.DataFrame([sp, sa, sr, sg]).T.rename(columns={0:'precipitation',1:'evapotranspiration',2:'runoff',3:'recharge'})
    df.index.rename('swsID', inplace=True)
    df.to_csv(ravenoutputdir+mdlprfx+'_SWS-waterbalance-{}.csv'.format(seriesname))    

In [73]:
gatherHRUtoSWS('current',dprecip,daet,dro,dbasflw,drecharge)
gatherHRUtoSWS('future',dprecipFuture,daetFuture,droFuture,dbasflwFuture,drechargeFuture)